In [ ]:
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
output_dir = "output/04/"
model_path = 'saved_models/112302best_model-epoch=00-val_ssim=0.5549.ckpt'

class CustomDataset(Dataset):
    def __init__(self, damage_dir, origin_dir, transform=None, use_masks=True):
        self.damage_dir = damage_dir
        self.origin_dir = origin_dir
        self.transform = transform
        self.use_masks = use_masks
        self.damage_files = sorted(os.listdir(damage_dir), key=lambda x: x.lower())
        self.origin_files = sorted(os.listdir(origin_dir), key=lambda x: x.lower())

    def __len__(self):
        return len(self.damage_files)

    def __getitem__(self, idx):
        damage_img_name = self.damage_files[idx]
        origin_img_name = self.origin_files[idx]

        damage_img_path = os.path.join(self.damage_dir, damage_img_name)
        origin_img_path = os.path.join(self.origin_dir, origin_img_name)

        damage_img = Image.open(damage_img_path).convert("RGB")
        origin_img = Image.open(origin_img_path).convert("RGB")

        if self.use_masks:
            input_data = get_input_image(damage_img_path, origin_img_path)
            mask = input_data['mask']
            # `mask`가 이미 텐서인지 확인하고 변환 처리
            if not isinstance(mask, torch.Tensor):
                mask = transforms.ToTensor()(mask)
        else:
            mask = torch.zeros((1, damage_img.size[1], damage_img.size[0]))

        if self.transform:
            damage_img = self.transform(damage_img)
            origin_img = self.transform(origin_img)

        return {'A': damage_img, 'B': origin_img, 'mask': mask, 'filename': damage_img_name}


# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dir = 'data/test_input'

# 테스트 데이터셋 생성
test_dataset = CustomDataset(damage_dir=test_dir, origin_dir=test_dir, transform=transform, use_masks=False)

# 테스트 DataLoader 생성
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,  # 배치 크기를 1로 설정하여 각 파일을 개별적으로 처리
    shuffle=False
)

# 모델 초기화
model = LitIRModel.load_from_checkpoint(
    checkpoint_path=model_path,
    model_1=model_1,
    model_2=model_2
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델을 평가 모드로 설정
model.eval()
model.to(device)

# 테스트 데이터로 예측 실행
os.makedirs(output_dir, exist_ok=True)
model.eval()  # 모델을 평가 모드로 설정

with torch.no_grad():
    for idx, batch in enumerate(test_dataloader):
        # 입력 데이터 준비 (RGB -> Grayscale 변환)
        inputs = torch.mean(batch['A'], dim=1, keepdim=True).to(device)  # [N, 1, H, W]
        
        # 모델 예측
        gray_restored, color_restored = model(inputs)  # 모델 예측

        # 파일명 가져오기
        filename = batch['filename'][0]  # 배치 크기가 1이므로 첫 번째 파일명만 사용

        # 예측된 이미지를 저장
        for i, result in enumerate(color_restored):
            result_img = (result.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)  # [C, H, W] -> [H, W, C]
            output_path = os.path.join(output_dir, filename)  # 원본 파일명과 동일하게 저장
            plt.imsave(output_path, result_img)
            print(f"Saved: {output_path}")